<a href="https://colab.research.google.com/github/npd2013/Mypublicmodels/blob/main/Experiment_6_1f_using_TFIDF_with_spell_check_dictionary__saluteremove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SETUP1:-  
-Single document text summarization 

-frequency based method for scoring sentences 

-preprocessing applied are knowledge based normalization,  stopword removal and stemming.
-NO CUE WORDS 

-automatic evaluation using rouge and compression ratio analysis.

note:results stored in another file 


Algorithm steps :- 

1) Preprocessing of text- 

    1.1)Cleaning 

    1.1.1)Removal of unnecessary symbols/charachters like html tags ,extra spaces,extra newlines etc.
         
 
 1.2) Line segmentation
 
    -using indicnlp with own segmenter  
    -Divide text into lines and find token words using own word segmenter.
 1.3) word segmentation
    -Divide the lines into words.
    -words can be separated by whitespace or characters like quotes or comma.  use the punctuation list to exclude from the word.Only extract valid words 

  1.4) Stopword removal 
 
       -basic set of 15 words removed from lines. 
   
 1.5) Stemming
    -if word is multiword expression,return  

    -make dictionary of valid words using indowordnet 
 
    -remove salutations from suffix like bhai,bahen etc
 
    -Correct wordspelling as per dictionary, if valid dictionary word. 

    -if not valid dictionary word

       -look up prefix and trim if match found

       -look up replacement and trim if match found

       - look up longest suffix rules matching and trim if match found.
    
    -if trimmed word size>=3 
            
            -    stemword = trimword
   
                   
               
       

2)Make vector space representation -based on term frequency

    2.1) Compute tfidf using sklearn module of python.

    2.2) For each sentence - 

     2.2.1) Compute sentence score= total tfidf score of all words in sentence divide by number of words in sentence.
     2.2.2) If it is first or last sentence , increase its weightage ten times.
  
  2.3) For each sentence -

     3.2.1) Compute the average normalized score  : 
          Current sentence score divide by total sentence score of text.
      

 4) //Find top "n" scored sentences.

 4.1) Set n= number of lines in gold summary. 0r "x" percent of original text.    
 4.2) Summary_length=n
 4.3) Sorted_sentences=sort(sentences) based on their scores.
    
 5) //Generate the summary -

    for sentences = 1 to gold_summary_length 
      - if sentence in gold_summary  
           put sentence in system_summary. 
  
6)// Evaluate the summary-

    6.1) use the Rouge score 
      - Get the precision ,recall and F-measure for rouge-1 ,rouge2 and rouge-LCS.   

    6.2) Find comression ratio of text
       6.2.1) sentence compression ratio
        = total sentences in system_summary/total sentences in source text 
        6.2.2) Word compression ratio
        =total words in system_summary/total words in source 
  


In [ ]:
#install needed packages 
#!pip install  gensim  #has word2vec model built, using english 

!pip install pyiwn
!pip install -U scikit-learn
!pip install rouge
!pip install indic-nlp-library


     |████████████████████████████████| 22.3MB 45.0MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
!cp "/content/drive/My Drive/Colab Notebooks/guj_final_preporcess-v6.py" "guj_final_preporcess.py" #The preprocessing modules designed
!cp "/content/drive/My Drive/Colab Notebooks/Evaluation.py" "Evaluation.py"   # the evaluation module designed
!cp "/content/drive/My Drive/Colab Notebooks/guj_final_postprocessing-v1.py" "post.py"   # Get the post processing module 


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import io
import csv
from collections import Counter
from Evaluation import *
from post import *
import pandas as pd 
from guj_final_preporcess import * #to get the required preporcessing modules -from self made pipeline
import pyiwn

2021-02-24:02:36:49,391 INFO     [helpers.py:20] Downloading IndoWordNet data of size ~31 MB...


[██████████████████████████████████████████████████]

2021-02-24:02:36:51,3 INFO     [helpers.py:43] Extracting /root/iwn_data.tar.gz into /root...


2021-02-24:02:36:52,231 INFO     [helpers.py:48] Removing temporary zip file from /root/iwn_data.tar.gz
2021-02-24:02:36:52,240 INFO     [helpers.py:51] IndoWordNet data successfully downloaded at /root/iwn_data


In [ ]:
from google.colab import files #upload the files from local machine on the colab environment 
uploaded = files.upload()

Saving story1.txt to story1.txt
Saving story1-gold.txt to story1-gold.txt


In [ ]:

#take all the needed inputs from user 
basename=input('Enter the basic name of file(without txt),other names wud be autogenerated  ')
filename=basename+'.txt'
gold_filename=basename+'-gold.txt'
#filename=input('Enter text file name to be summarized ') #Ex: file1.txt
analysefile_name=filename.replace('.txt','analysis-TFIDF .txt')# to store the data for later analysis . Example- file1-results.txt 
features_filename=filename.replace('.txt','TFIDF-Features.csv') # file-1-features.csv
#gold_filename=input("Enter name of file containing gold summary  ") #example file1-gold.txt ,assumes the name is having pattern ending with gold.txt 
summary_filename=filename.replace('gold.txt','-TFIDF+ALL_systemsummary.txt') # example file1-gold-system-summary.txt


Enter the basic name of file(without txt),other names wud be autogenerated  story1


In [ ]:
#read the input original file . 


guj_clean_html(filename,filename)# text cleaning remove the html tags and overwrite the original file 
guj_clean_html(gold_filename,gold_filename)# clean the html tags from gold file 

#Display the given data in input files 
with io.open(filename,'r',encoding='utf-8') as f: 
		  lines=f.read()#get all the lines from file
analyzedfile_obj=open(analysefile_name,mode='w',encoding='UTF-8')
analyzedfile_obj.write("\n\n The original file is :------\n")
analyzedfile_obj.write(lines)

with io.open(gold_filename,'r',encoding='utf-8') as f: 
		  lines=f.read()#get all the lines from file

analyzedfile_obj.write("\n\n The gold summary provided by expert  is -->\n")    
analyzedfile_obj.write(lines)




# now find the statistics of originally given informations 
originalcorpora,totaloriginalsentences=guj_corpus_generate(filename) #make corpus of original data 
goldcorpora,totalgoldcorpora=guj_corpus_generate(gold_filename) #make corpus of gold 

analyzedfile_obj.write("\n\n STATISTICS------>")
analyzedfile_obj.writelines("\n Original file sentences(RAW) - {0}\n".format(totaloriginalsentences))
analyzedfile_obj.writelines("\n gold file sentences(RAW)- {0}\n".format(totalgoldcorpora))

###################################################################
########################  Step-1 ##################################
#####################  cleaning phase  ########################

#doing cleaning of file 
#remove newlines and other unnecessary data and clean it for feature extraction 
#to replace the data as per our format, replacing abbreviations dot with '*'
normfile="temp1.txt"
norm_gold_file="temp2.txt"

guj_clean_text(filename,normfile) 
guj_clean_text(gold_filename,norm_gold_file)


#Now display the stats and make corpora after cleaning file and removing new lines . 
normcorpora,totalnormsentences=guj_corpus_generate(normfile)  
goldcorpora,totalnormgoldsentences=guj_corpus_generate(norm_gold_file) 


analyzedfile_obj.write(" Segmented Lines in source -{0} \n ".format(totalnormsentences))
analyzedfile_obj.write(" Segmented Lines in gold -{0}\n ".format(totalnormgoldsentences))

###################################################################
########################  Step-2 ##################################
#####################  Normalization and tokenization  for feature extraction  ########################

cleanfile="file.txt"
mydictionary=guj_makedictionary()
guj_stopwordremoval(normfile,cleanfile) #remove stopwords
guj_stemmer_new(cleanfile,cleanfile,mydictionary)




#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#####


#Now working on the modified file to get various features 

corpora,totalsent=guj_corpus_generate(cleanfile) #divide lines into sentences-
analyzedfile_obj.write(" Lines in source after all processing -{0} \n ".format(totalsent))

sent_pos_list=[] #to store the sentence position
sent_words_count=[]  # to store the total words in each sentence 
sent_number=0
for s in corpora:
  sent_pos_list.append(sent_number)
  sent_number+=1

  

# to generate vector based on frequency 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer = guj_mytokenizer,min_df=0.15,max_df=0.85) #frequency of words with atleast 15% frequency
#vectorizer = TfidfVectorizer(tokenizer = guj_mytokenizer)
cwm = vectorizer.fit_transform(corpora) #vectorize the corpus of input file


tokens = vectorizer.get_feature_names() #get the names of words used in vector

score_list=[]
counts=cwm.sum(axis=1) #sums the sentence wise words' tfidf scores
scorelist=list(counts.flat)#get the list of sentencescores 


analyzedfile_obj.write("\n\n****The TFIDF  based vector is formed ****")
analyzedfile_obj.write('\n The dimension of vector matrix generated is -\n {0} '.format(cwm.shape))

 
 
list_of_tuples = list(zip(sent_pos_list,corpora, cwm.toarray()[0]))  
#generate a dataframe to store the data 
import pandas as pd 
df=pd.DataFrame(list_of_tuples, columns=['sent_pos','sentence','frequency_vector'])
df.to_csv(features_filename) 


##########################################################
################# ######Step3#############################
################ Compute sentence scores #################
##########################################################

score_list=[] #to store sentencewise scores original 
totwords=[]
for s in corpora:
   words=guj_mytokenizer(s)
   #print("number of words in sentence are",len(words))
   totwords.append(len(words))
newscorelist=[]

list_of_tuples = list(zip(sent_pos_list,corpora, cwm.toarray(),scorelist,totwords))  
df=pd.DataFrame(list_of_tuples, columns=['sent_pos','sentence','feature_vector','totalsentcount','totalwordsinsent'])

df['sent_score']=df['totalsentcount']/df['totalwordsinsent']



df.to_csv(features_filename) 




In [ ]:

rougescorefile="Rougeresults-all-TF.csv"
rougescorefilehandle=open(rougescorefile,mode='a',encoding="utf-8") #to store only the rouge scores for later analysis 

resultvar = csv.writer(rougescorefilehandle, lineterminator='\n')		#to store new row for each f  

##########################################################
################# ######Step4#############################
################ Find top sentences  #################
##########################################################
#rank and sort the sentences  based on individual sentences score 

optimalratio=(totalnormgoldsentences/totalnormsentences)*100 #ideal ratio as per gold text given 
optimalratio = float(f"{optimalratio :.2f}")
data=df.sort_values(["sent_score"],ascending=False,kind='quicksort')
resultlist=[]#to store the rouge score data
for percent in [optimalratio]:
  resultlist.append("Extraction ratio"+str(percent)+",")
  analyzedfile_obj.write("\n\nThe method of summary is *** {0}% *** of original text".format(percent))
  n= round(totalnormsentences*int(percent)/100)
  top_sent=data.iloc[0:n,1:2]  
 # print(top_sent.head())
  toplist=list(top_sent.sentence)
 
  analyzedfile_obj.write("\n\n List of top sentences are as follows-{0} ".format(toplist))
  ##########################################################
################# ######Step6#############################
################ Generate summary  #################
##########################################################
  summary=[]
  not_in_summary=[]
  sentence_number=0
  for sent in corpora:  #take one by one the sentences considered in feature vector 
    if sent in toplist: #if sentence is in topranked sentences then store its number so its original line can be fetched from original corpus
        summary.append(sentence_number)
    else:
        not_in_summary.append(sentence_number)
    sentence_number=sentence_number+1

  analyzedfile_obj.write("\n\n ***** The summary geneated has the following numbered sentences ---->>\n {0}".format(summary))
  analyzedfile_obj.write("\n\n ***** The sentences not included in summary are these numbers---->>{0}".format(not_in_summary))
  
  counter=0 #now fetching the original lines to be copied in final summary. 
  summary_lines=[]
  for sent in normcorpora:
    if counter in summary:
      summary_lines.append(sent)   
    counter=counter+1

 
  #store the summary in file
  summary_filename='system-temp-{0}.txt'.format(str(percent))
    #to give name tosummary file for storing percent based results
  #format is  original file name -system-<percent> Ex:- news1-system-50
  
  with io.open(summary_filename,'w',encoding='utf-8') as w: 
    for i in summary_lines:
      w.write(i)
  
  ###########step5#############
  ########### Post Processing ######
  resultfilename="system_summary_TF+stopwordr+stem_"+filename
  
  post_process(summary_filename,resultfilename)#do the post processing and write in result file
################# ######Step6#############################
################ Evaluate summary  #################
  guj_remove_newlines(norm_gold_file,norm_gold_file)
  guj_remove_newlines(summary_filename,summary_filename)
  rougescores=evalate_files(norm_gold_file,summary_filename)#get the evaluation of generated summary after post processing based on rouge score
  for key in rougescores.keys():
     resultlist.append("%s,%s"%(key,rougescores[key]))
  resultvar.writerow(resultlist)   
  analyzedfile_obj.write("\n **********************Rouge scores based analysis for {0} % extraction is as follows*********************** ".format(percent))
  analyzedfile_obj.write("\n\n The rouge scores on summary file are \n {0}".format(rougescores,2)) #store the results 

  basicfile="original.txt"
  #guj_clean_text(filename,basicfile) # Get the original file in cleaned form for computing compression ratio. 
  guj_remove_newlines(filename,basicfile) 
  
  sent_compression,word_compression,totaloriginalsentences,totaloriginalwords,totalsumsentences,totalsumwords=compression_statistics(summary_filename,basicfile) #to store the compression ratio analysis
  analyzedfile_obj.write("\n **********************Compression Analysis at {0} % extraction is as follows *********************** ".format(percent))
  analyzedfile_obj.write("\n Total sentences in original text were {0} ".format(totaloriginalsentences)) 
  analyzedfile_obj.write("\n Total sentences in summary text are {0} ".format(totalsumsentences))
  analyzedfile_obj.write("\n Sentences compression ratio is {0} \n".format(sent_compression,2))
  analyzedfile_obj.write(" \n\n Total words in original text were {0} ".format(totaloriginalwords))
  analyzedfile_obj.write(" \n total words in summary text are {0} ".format(totalsumwords))
  analyzedfile_obj.write("\n Words compression ratio is {0} ".format(word_compression,2)) 


{'rouge-1': {'f': 0.5911949635851915, 'p': 0.642369020501139, 'r': 0.5475728155339806}, 'rouge-2': {'f': 0.46008402864531106, 'p': 0.5, 'r': 0.4260700389105058}, 'rouge-l': {'f': 0.5807560087742825, 'p': 0.6282527881040892, 'r': 0.5399361022364217}}


In [ ]:
analyzedfile_obj.close()

rougescorefilehandle.close()